In [1]:
from scipy.stats import chi2_contingency
from scipy.stats import ttest_ind
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np


# Test hypotéz

In [ ]:
# Získání záznamů ze souboru
total_df = pd.read_pickle("accidents.pkl.gz")

## Hypotéza 1
Na silnicích první třídy se při nehodách umíralo se stejnou pravděpodobností jako dálnicích.

Pro tuto hypotézu je doporučený test X^2. Pro tento test si připravíme 1 dataset, který obsahuje rozdělení typů komunikací v závislosti na počtu fatálních a nefatálních nehod.
fatální nehody jsou ty, při kterých umřela alespoň jedna osoba.

In [4]:
# Získání záznamů nehod pouze z dálnic a silnic první třídy
df_h1 = total_df.loc[(total_df["p36"] == 0)|(total_df["p36"] == 1)]

# Vytvoření dataframu pouze se sloupci s typem silnic a sloupcem obsahující 
# jestli byla nehoda fatalní, nebo nebyla.
df_h1 = pd.DataFrame({
    "type" : df_h1["p36"],
    "dead" : df_h1["p13a"] >0
}).dropna()

# Vytvoření cross tabulky z dataframu "df_h1"
df_h1 = pd.crosstab(df_h1["type"], df_h1["dead"])

print(df_h1)

dead  False  True
type             
0     24293   166
1     78618   911


In [5]:
# Aplikace testu x^2 na "df_h1" cross tabulku
stat, p, dof, expected = chi2_contingency(df_h1)

print("P hodnota je: ", p)
print("Očekávaný počet fatálních nehod na silnicíh 1. třídy je:", expected[1][1])

P hodnota je:  3.6067450279444316e-10
Očekávaný počet fatálních nehod na silnicíh 1. třídy je: 823.6790110397353


Z p hodnoty je zřejmé, že je hypotéza zamítnuta. Data nekorelují. Na silnicích první třídy se neumíralo se stejnou pravděpodobností jako na dálnicích.

Z testu je také zřejmé, že očekávaná hodnota fatálních nehod na dálnicích by měla být 823.67.... Ale reálný počet fatálních nehod na dálnicích je roven 911.

## Hypotéza 2
Při nehodách vozidel značky Škoda je škoda na vozidle nižší než při nehodách vozidel Audi.

Pro vyhodnocení hypotézy vyuýžijeme T-test, kterým můžeme porovnat 2 datasety poškození na vozidlech.

In [7]:
# Získání záznamů nehod pouze vozidel Audi a Škodauložených v separovaných dataframech
audi_df_h2 = total_df.loc[total_df["p45a"] ==2, "p53"]
skoda_df_h2 = total_df.loc[total_df["p45a"] ==39, "p53"]

# Aplikace t testu na porovnání 2 skupin záznamů nehod
result = ttest_ind(audi_df_h2, skoda_df_h2)

print(result)

Ttest_indResult(statistic=-39.12749433594028, pvalue=0.0)


P hodnota je na dostatečné hladině významnosti, takže hypotézu nezamítáme. Výsledná statistika T testu je kladná. To potvrzuje hypotézu, že při nehodách značky Škoda je škoda na vozidle nižší než při nehodách vozidel Audi.